In [1]:
# Подключаем необходимые библиотеки и делаем общие настройки
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.options.mode.copy_on_write = True

from tqdm import tqdm
from datetime import datetime
from external_functions import new_feature_by_user, new_feature_by_item, get_model_input 
import json

In [2]:
# Загрузим данные из events.csv 
df_events = pd.read_csv('events.csv')

In [3]:
# (1) Удалим дубликаты, если они есть
df_events.drop_duplicates(inplace=True)

In [4]:
# Статистика по датасету
print('Записей в рабочем датасете:', df_events.shape[0])
print('Уникальных пользователей:', df_events['visitorid'].nunique())
print('Уникальных товаров:', df_events['itemid'].nunique())

Записей в рабочем датасете: 2755641
Уникальных пользователей: 1407580
Уникальных товаров: 235061


In [5]:
# (2) Преобразуем временной признак в долее удобный для дальнейшей обработки формат
df_events['timestamp'] = pd.to_datetime(df_events['timestamp'], unit='ms')

In [6]:
# (3) Удалим признак transactionid, как не информативный в нашем случае
df_events.drop(['transactionid'], axis=1, inplace=True)

In [7]:
# (4) В признаке event сделаем продажи и корзину как 1
event_dic = {'view':0, 'addtocart':1, 'transaction':1}
df_events['event'] = df_events['event'].apply(lambda x: event_dic[x])
df_events['event'].value_counts()

0    2664218
1      91423
Name: event, dtype: int64

In [8]:
# (5) Подготовим части датасета на основе временного признака, для дальнейшего формирования из них выбороки для модели
start = min(df_events['timestamp'])
end = max(df_events['timestamp'])
interval = end - start

df_1 = df_events[df_events['timestamp'] < (end - interval/2)]
df_2 = df_events[df_events['timestamp'] >= (end - interval/2)]

df_target_user = set(df_1['visitorid']) & set(df_2['visitorid'])

In [9]:
# (6) Формируем выборки с новыми признаками для частей датасета раздельно по пользователям и товарам (1 ч. 5 мин)
df_1_u = new_feature_by_user(df_1)
df_1_i = new_feature_by_item(df_1)

100%|█████████████████████████████████████████████████████████████████████████| 176735/176735 [12:49<00:00, 229.78it/s]


In [10]:
# (7) Сформируем выборки, необходимые для передачи модели
X_df, y_df, query_list_df = get_model_input(df_1_u, df_1_i, df_2, df_target_user)

In [11]:
# Сохраним входные данные для предсказаний во внешний файл
X_df.to_csv('data_to_predict.csv')

In [12]:
# Сохраним данные c реальными действиями пользователей во внешний файл
y_df.to_csv('data_to_metric.csv')

In [13]:
# Выделим Топ-3 товаров для добавки в итог рекомендаций, при недостаке значений в предсказании
top3_items = list(df_events[df_events['event'] == 1]['itemid'].value_counts().index[:3])

In [14]:
with open('top3_items.ini','w') as f:
    json.dump(top3_items, f)